In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
#raw = importData('E29_CHECKIN1_MACROANALYSIS_V1.csv').replace("No Response", np.nan)

In [3]:
def importData(csv):
    data = pd.read_csv(csv)
    return data

#Break down goal setting and team process into: goals, support, communication, work allocation, team role, and enjoyment
#Then, find the magnitude of such difficulties for each team
def getCategoryDifferences(df, teamNumberCol):
    #Subset df to just goal setting and team process quantitative questions
    columnsOfInterest = [teamNumberCol, "SharedGoalTeamAvg", "SharedGoalClassAvg", 
                         "IndvGoalTeamAvg", "IndvGoalClassAvg", "SupportTeamAvg", 
                         "SupportClassAvg", "CommunicateTeamAvg", "CommunicateClassAvg", 
                         "WorkAllocTeamAvg", "WorkAllocClassAvg", "RoleTeamAvg", 
                         "RoleClassAvg", "EnjoyTeamAvg", "EnjoyClassAvg", 
                        'ConflictResTeamAvg', 'ConflictResClassAvg', 'ConflictResClassStDev',
                        'EfficiencyTeamAvg', 'EfficiencyClassAvg', 'EfficiencyClassStDev',
                        'ExpressionTeamAvg', 'ExpressionClassAvg', 'ExpressionClassStDev']
    temp = df[columnsOfInterest]
    
    #Collapse rows so that 1 row = 1 team
    temp = temp.drop_duplicates()
    
    #Calculate all averages (= team's cumulative deviation from class average)
    temp['goalsDiff'] = (temp["SharedGoalTeamAvg"] + temp["IndvGoalTeamAvg"]) - (temp["SharedGoalClassAvg"] + temp["IndvGoalClassAvg"])
    temp['supportDiff'] = temp["SupportTeamAvg"] - temp["SupportClassAvg"]
    temp['communicateDiff'] = temp["CommunicateTeamAvg"] - temp["CommunicateClassAvg"]
    temp['workAllocationDiff'] = temp["WorkAllocTeamAvg"] - temp["WorkAllocClassAvg"]
    temp['teamRoleDiff'] = temp["RoleTeamAvg"] - temp["RoleClassAvg"]
    temp['enjoyDiff'] = temp["EnjoyTeamAvg"] - temp["EnjoyClassAvg"]
    temp['conflictResDiff'] = temp["ConflictResTeamAvg"] - temp["ConflictResClassAvg"]
    temp['efficiencyDiff'] = temp['EfficiencyTeamAvg'] - temp['EfficiencyClassAvg']
    temp['expressionDiff'] = temp['ExpressionTeamAvg'] - temp['ExpressionClassAvg']
    temp['overallDiff'] = round(temp['goalsDiff'] + temp['supportDiff'] + temp['communicateDiff'] + temp['workAllocationDiff'] + temp['teamRoleDiff'] + temp['enjoyDiff'] + temp['conflictResDiff'] + temp['expressionDiff'] + temp['efficiencyDiff'], 2)
    
    tempRanked = temp.sort_values(teamNumberCol, ascending=True).reset_index(drop=True)
    tempRanked = tempRanked[[teamNumberCol, 'overallDiff', 'goalsDiff', 'supportDiff', 
                             'communicateDiff', 'workAllocationDiff', 'teamRoleDiff', 'enjoyDiff',
                            'conflictResDiff', 'efficiencyDiff', 'expressionDiff']]
    
    return tempRanked

#Assign each team a percentile (score) based on their overall difference from the class average. A difference of 0 designates a 50 (median)
def getTeamPercentiles(df, teamNumberCol):

    arr = np.array(df['overallDiff'])
    df["percentile"] = [round(stats.percentileofscore(arr, x), 2) for x in arr]

    return df
#Assign a category according to each team's percentile
def categorizePercentiles(df, teamNumberCol):
    arr = np.array(df['percentile'])
    cats = []
    #print(arr)
    for score in arr:
        if score >= 75:
            cats.append("Exceptional")
        elif score >= 50 and score < 75:
            cats.append("Strong")
        elif score >= 20 and score < 50:
            cats.append("Standard")
        elif score >= 10 and score < 20:
            cats.append("Weak")
        elif score >= 0 and score < 10:
            cats.append("Needs Attention")
        else:
            cats.append("Team did not respond")
    #print(cats)
    df["percentileCategory"] = cats
    return df

In [5]:
E29_CHECKIN02 = pd.read_csv("E29_CHECKIN04_MACROANALYSIS_V1.csv")

In [6]:
diffs = getCategoryDifferences(E29_CHECKIN02, 'TeamNumber')

In [7]:
percentiles = getTeamPercentiles(diffs, 'TeamNumber')

In [8]:
scored = categorizePercentiles(percentiles, 'TeamNumber')

In [9]:
scored.head()

TeamNumber  overallDiff  goalsDiff  supportDiff  communicateDiff  \
0           1          4.3        0.5          0.2              0.5   
1           2          3.7        0.7          0.5              0.4   
2           3         -7.8       -1.0         -0.5             -0.6   
3           4          8.1        1.3          0.8              1.0   
4           5          5.5        0.7          0.5              0.7   

   workAllocationDiff  teamRoleDiff  enjoyDiff  conflictResDiff  \
0                 0.8           0.0        0.6              0.6   
1                 0.7           0.0        0.3              0.3   
2                -0.7          -1.0       -1.4             -0.4   
3                 1.3           0.6        0.9              0.9   
4                 1.0           0.3        0.6              0.6   

   efficiencyDiff  expressionDiff  percentile percentileCategory  
0             0.7             0.4       60.00             Strong  
1             0.4             0.4       53.33             Strong  
2            -1.6            -0.6       16.67               Weak  
3             0.6             0.7       96.67        Exceptional  
4             0.7             0.4       68.33             Strong

In [10]:
roster = pd.read_csv("E29_CHECKIN04_ROSTER_V2.csv")
roster = roster[["TeamName", "TeamNumber"]].drop_duplicates().reset_index()
macro = pd.read_csv("E29_CHECKIN04_MACROANALYSIS_V1.csv")
scored = scored

In [11]:
scored

TeamNumber  overallDiff  goalsDiff  supportDiff  communicateDiff  \
0            1          4.3        0.5          0.2              0.5   
1            2          3.7        0.7          0.5              0.4   
2            3         -7.8       -1.0         -0.5             -0.6   
3            4          8.1        1.3          0.8              1.0   
4            5          5.5        0.7          0.5              0.7   
5            6          6.9        1.1          1.0              0.5   
6            7        -13.6       -2.3         -1.5             -0.8   
7            8          2.2        1.5          0.5              0.4   
8            9          6.4        0.9          0.7              0.7   
9           10         10.2        1.7          1.0              1.2   
10          11          1.3        0.9          0.2              0.2   
11          12          0.7        0.0         -0.2             -0.1   
12          13          6.1        1.1          0.7              0.7   
13          14          7.8        1.7          0.7              0.9   
14          15         -7.1       -1.5         -1.2             -1.0   
15          16        -11.3       -2.9         -1.6             -1.4   
16          17          4.3        0.9          0.7             -0.1   
17          18          5.5        0.9          0.6              0.6   
18          19         -7.9       -1.5         -0.6             -1.0   
19          20         -6.7       -1.9         -0.6             -1.0   
20          21         -2.9       -0.3          0.0             -0.4   
21          22         -3.8       -0.7         -0.3             -0.3   
22          23        -12.3       -1.8         -1.0             -1.8   
23          24          4.3        0.7          0.6              0.8   
24          25         -6.8       -0.6         -1.5             -0.5   
25          26          7.3        1.3          0.8              0.8   
26          27         -3.7        0.5          0.4             -0.6   
27          28          7.2        1.2          0.8              1.0   
28          29         -7.6       -1.3         -0.7             -1.0   
29          30          6.3        0.7          0.6              0.8   

    workAllocationDiff  teamRoleDiff  enjoyDiff  conflictResDiff  \
0                  0.8           0.0        0.6              0.6   
1                  0.7           0.0        0.3              0.3   
2                 -0.7          -1.0       -1.4             -0.4   
3                  1.3           0.6        0.9              0.9   
4                  1.0           0.3        0.6              0.6   
5                  0.8           0.5        1.1              0.4   
6                 -1.7          -1.0       -1.7             -1.7   
7                 -0.3          -0.2        0.1              0.1   
8                  0.8           0.0        0.9              0.8   
9                  1.2           0.8        1.1              1.1   
10                 0.0           0.5       -0.1             -0.7   
11                 0.3          -0.2        0.3              0.3   
12                 0.8           0.3        0.6              0.6   
13                 0.5           0.6        0.9              0.9   
14                -1.0          -0.2       -0.9             -0.7   
15                -1.3          -0.4       -0.7             -2.3   
16                 0.5           0.1        0.8              0.6   
17                 0.9           0.2        0.7              0.7   
18                -1.3          -0.4       -1.1             -0.7   
19                -1.5          -0.6        0.1             -0.5   
20                -0.7          -0.4        0.3             -0.3   
21                -0.7          -0.4       -0.2             -0.2   
22                -1.5          -0.5       -1.2             -1.4   
23                 0.5           0.0        0.5              0.5   
24                -1.3           0.0       -1.6             -0.2   
25 

In [12]:
final = macro.merge(scored, on='TeamNumber').merge(roster, on='TeamNumber')

In [13]:
final

Unnamed: 0              FullName                        Email  \
0             0           Arush Godha     arush.godha@berkeley.edu   
1             1             Aaron Lee    aaronlee5702@berkeley.edu   
2             2        Naomi Nogueira  naomi_nogueira@berkeley.edu   
3             3             Khant Nyi            knyi@berkeley.edu   
4             4  Brandon Ramos-Bailon      bramos6837@berkeley.edu   
..          ...                   ...                          ...   
167         167          Pablo Brunet         pbrunet@berkeley.edu   
168         168          Eilyn Garcia     garciaeilyn@berkeley.edu   
169         169    Tiffelyn Kurniawan   tifkurnia2004@berkeley.edu   
170         170             Eric Lind          elind1@berkeley.edu   
171         171          Brandon Wong         wongton@berkeley.edu   

     TeamNumber  TeammateNumber  \
0             1               1   
1             1               2   
2             1               3   
3             1               4   
4             1               5   
..          ...             ...   
167          30               2   
168          30               3   
169          30               4   
170          30               5   
171          30               6   

     Our team is clear about the shared goals for our work together  \
0                                                  7.0                
1                                                  6.0                
2                                                  6.0                
3                                                  7.0                
4                                                  6.0                
..                                                 ...                
167                                                7.0                
168                                                7.0                
169                                                7.0                
170                                                6.0                
171                                                6.0                

     SharedGoalTeamAvg  SharedGoalClassAvg  SharedGoalClassStDev  \
0                  6.5                 6.2                   1.0   
1                  6.5                 6.2                   1.0   
2                  6.5                 6.2                   1.0   
3                  6.5                 6.2                   1.0   
4                  6.5                 6.2                   1.0   
..                 ...                 ...                   ...   
167                6.6                 6.2                   1.0   
168                6.6                 6.2                   1.0   
169                6.6                 6.2                   1.0   
170                6.6                 6.2                   1.0   
171                6.6                 6.2                   1.0   

     We each know about one another's individual goals for our work together  \
0                                                  7.0                         
1                                                  6.0                         
2                                                  6.0                         
3                                                  6.0                         
4                                                  6.0                         
..                                                 ...                         
167                                                7.0                         
168                                                7.0                         
169                                                7.0                         
170                                                5.0                         
171                                                6.0                         

     ...  workAllocationDiff  teamRoleDiff  enjoyDiff  conflictResDiff  \
0    ...                 0.8          

In [14]:
final.to_csv("E29_CHECKIN04_FULL_ANALYSIS_V1.csv")